# Reator

![reator](./reator.png)


**Equacionamento**:

$\frac{d h}{dt} =  F_1 + F_2 - c_v \cdot \sqrt{h}$

$\frac{d c_\text{B}(t)}{d t} = (Cb_\text{1,in}-c_\text{B})\cdot\frac{F_1}{h}+(Cb_{2,\text{in}}-c_\text{B})\cdot\frac{F_2}{h}-k_1\cdot\frac{Cb}{(1+k_2\cdot c_\text{B})^{1.5}}$

$\frac{d c_\text{C}(t)}{d t} = (-c_\text{C})\cdot\frac{F_1}{h}+(-c_\text{C})\cdot\frac{F_2}{h}+k_1\cdot\frac{Cb}{(1+k_2\cdot c_\text{B})^{1.5}}$


Reações: 

$B \rightarrow C$

**Variáveis**:

- A área da seção transversal do reator é unitária

- $h$ -> nível do reator

- $F_1$ -> vazão de alimentação 1 (m$^3$/s)

- $F_2$ -> vazão de alimentação 2 (m$^3$/s)

- $Cb_\text{1,in}$ -> concentração de B na corrente de alimentação 1 do reator (gmol/m$^3$)

- $Cb_\text{2,in}$ -> concentração de B na corrente de alimentação 2 do reator (gmol/m$^3$)

- $c_v$ -> coeficiente de descarga

- $C_\text{B}$ -> concentração de B no reator (gmol/m$^3$)

- $C_\text{C}$ -> concentração de C no reator (gmol/m$^3$)

- $k_1$ -> parâmetro associado à taxa de conversão de B em C 

- $k_2$ -> parâmetro associado à taxa de conversão de B em C 


Adaptado de:

Ali, E., & Zafiriou, E. (1993). Optimization-based tuning of nonlinear model predictive control with state estimation. Journal of Process Control, 3(2), 97–107. https://doi.org/10.1016/0959-1524(93)80005-V

In [1]:
# Importação de pacotes
import serial

from numpy import linspace, array, hstack

## Pacotes que permitem manipular como os gráficos aparecem neste notebook
from matplotlib.pyplot import figure, tight_layout
from IPython import display
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots

In [2]:
# Gráfico

def grafico(h,Cb,Cc,u1,u2,Cb1,Cb2,instanteTempo):
    
    fig = figure(figsize=(8,4))
    
    ax = fig.add_subplot(1, 3, 1)
    ax.plot([0, 0], [0, 4], 'k-', [1, 1], [0, 4], 'k-')
    ax.plot([0, 1],[float(h), float(h)],'b-')
    ax.set_ylabel('h')
    ax.set_ylim(0,4)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(True)
    
    ax = fig.add_subplot(1, 3, 2)
    ax.plot([0, 0], [0, 8], 'k-',[1, 1], [0, 8], 'k-')
    ax.plot([0, 1], [float(Cb), float(Cb)], 'b-')
    ax.set_ylabel('Cb')
    ax.set_ylim(0,8)
    ax.get_xaxis().set_visible(False)
    
    ax = fig.add_subplot(1, 3, 3)
    ax.plot([0, 0], [0, 8], 'k-',[1, 1], [0, 8], 'k-')
    ax.plot([0, 1], [float(Cc), float(Cc)], 'b-')
    ax.set_ylabel('Cc')
    ax.set_ylim(0,8)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(True)
    
    ax.text(1.2, 5, 'Tempo: {:.2f} s'.format(instanteTempo), fontsize=14, verticalalignment='top',)
    ax.text(1.2, 4, 'u1: {:.2f} l/s | u2: {:.2f} l/s'.format(float(u1), float(u2)), fontsize=14, verticalalignment='top',)
    ax.text(1.2, 1, 'Cb1: {:.2f} gmol/l | Cb2: {:.2f} gmol/l'.format(float(Cb1),float(Cb2)), fontsize=14, verticalalignment='top',)
    
    tight_layout()
    
def grafico_tendencia(h,Cb,Cc,u1,u2,Cb1,Cb2,namostra):
    
    tempo = linspace(0,(namostra-1)*Ts,namostra)

    fig = figure(figsize=(10,5))
    axes = fig.add_subplot(4,1,1)
    axes.plot(tempo,h,'.-')
    axes.set_ylabel('$h$/ l')
    axes.get_xaxis().set_visible(False)

    axes = fig.add_subplot(4,1,2)
    axes.plot(tempo,Cb,'.-')
    axes.set_ylabel('$Cb$ / gmol/m$^3$')
    axes.get_xaxis().set_visible(False)

    axes = fig.add_subplot(4,1,3)
    axes.plot(tempo,Cc,'.-')
    axes.set_ylabel('$Cc$/ gmol/m$^3$')
    axes.set_xlabel('tempo /s')

    fig = figure(figsize=(10,5))

    axes = fig.add_subplot(4,1,1)
    axes.plot(tempo,u1,'.-')
    axes.set_ylabel('$F1$ / $m^3/s$')
    axes.get_xaxis().set_visible(False)

    axes = fig.add_subplot(4,1,2)
    axes.plot(tempo,u2,'.-')
    axes.set_ylabel('$F2$ / $m^3/s$')
    axes.get_xaxis().set_visible(False)

    axes = fig.add_subplot(4,1,3)
    axes.plot(tempo,Cb1,'.-')
    axes.set_ylabel('$Cb_{1,in}$ / gmol/m$^3$')
    axes.set_xlabel('tempo /s')
    axes.get_xaxis().set_visible(False)

    axes = fig.add_subplot(4,1,4)
    axes.plot(tempo,Cb2,'.-')
    axes.set_ylabel('$Cb_{2,in}$ / gmol/m$^3$')
    axes.set_xlabel('tempo /s')

In [13]:
ser = serial.Serial('/dev/ttyUSB0', 9600)  # Establish the connection on a specific port

nsim = 100 # Quantidade total de amostras

Ts = 0.7 # Período de amostragem [s]

# Condições iniciais

h0 = 1.
Cb0 = 4.
Cc0 = 0.

# Variáveis exógenas
u1 = [0.1]*300 # m3/s
[u1.append(0.5) for it in range(200)]
[u1.append(0.1) for it in range(20)]

u2 = [0.1]*300# m3/s
[u2.append(0.3) for it in range(200)]
[u2.append(1) for it in range(100)]

Cb1 = [24.9]*300 # Concentração carga gmol/m3
[Cb1.append(1.) for it in range(30)]
[Cb1.append(2.9) for it in range(50)]

Cb2 = [0.1]*300 # Concentração carga gmol/m3
[Cb2.append(1.) for it in range(30)]
[Cb2.append(2.9) for it in range(50)]


# Iniciando listas
h = []
Cb = []
Cc = []
u1_ = []
u2_ = []
Cb1_ = []
Cb2_ = []

# Comunicação
x = ser.readline()
print(x)

#Escrevendo condicoes inicias
ser.write(bytearray('{:.2f};{:.2f};{:.2f}'.format(h0,Cb0,Cc0),'ASCII'))

# Simulando...
for it in range(nsim):
    
    ser.write(bytearray('{:.1f};{:.2f};{:.2f};{:.2f};{:.2f};{:.2f}\n'.format(1,u1[it],u2[it],Cb1[it],Cb2[it],Ts),'ASCII'))

    x = ser.readline().decode("utf-8")
    
    data = x.split('&')

    h.append(float(data[0]))
    Cb.append(float(data[1]))
    Cc.append(float(data[2])) 
    u1_.append(float(data[3])) 
    u2_.append(float(data[4]))
    Cb1_.append(float(data[5]))
    Cb2_.append(float(data[6]))
    
    #print(end-start)
    
#     # mostrando a figura
    #grafico(*data,it*Ts)
    grafico_tendencia(h,Cb,Cc,u1_,u2_,Cb1_,Cb2_,it+1)
    display.clear_output(wait=True)
    show_inline_matplotlib_plots()
    
    
ser.close() # Fechando a conexão com o arduino através da porta serial

KeyboardInterrupt: 

In [12]:
ser.close()

In [10]:
Cc

[0.04,
 -0.11,
 0.13,
 0.17,
 0.21,
 0.25,
 0.29,
 0.33,
 0.37,
 0.41,
 0.24,
 0.48,
 0.51,
 0.35,
 0.58,
 0.61,
 0.64,
 0.48,
 0.51,
 0.74,
 0.77,
 0.79,
 0.62,
 0.85,
 0.88,
 0.7,
 0.93,
 0.96,
 0.98,
 0.81,
 1.03,
 0.85,
 0.88,
 1.1,
 1.12,
 1.14,
 1.17,
 1.19,
 1.01,
 1.03,
 1.05,
 1.07,
 1.09,
 1.31,
 1.32,
 1.14,
 1.16,
 1.38,
 1.2,
 1.21,
 1.43,
 1.44,
 1.46,
 1.28,
 1.29,
 1.31,
 1.32,
 1.33,
 1.35,
 1.56,
 1.38,
 1.59,
 1.6,
 1.41,
 1.63,
 1.64,
 1.45,
 1.46,
 1.67,
 1.69,
 1.7,
 1.71,
 1.72,
 1.53,
 1.74,
 1.75,
 1.56,
 1.57,
 1.78,
 1.79,
 1.8,
 1.61,
 1.81,
 1.62,
 1.83,
 1.84,
 1.85,
 1.66,
 1.66,
 1.87,
 1.68,
 1.89,
 1.69,
 1.7,
 1.71,
 1.91,
 1.92,
 1.93,
 1.93,
 1.74]